Go to https://drive.google.com/drive/folders/1Pe6D713L9S0GWwb_XzeLXAeNZOrBqZaN?usp=sharing and click add shortcut to drive. This will add the data required for this assignment to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>

Caution: Since this is real language on Twitter and deals with world events, some of it could be disturbing. In the later section of the course, we will deal with ethics and what is appropriate and what is not.

First, let's read the tweets from the Google drive and print a few lines from the file.



Run the below code snippet. It will generate a URL which generates an authorization code. Enter it below to give Colab access to your Google drive.

**Note:** Copy button may not work. Try copying the authorization code manually.

In [19]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


When you run the `ls` command below, you should see the files in the tweets folder.




In [20]:
!ls "/content/drive/My Drive/tweets"

20000_tweets.jsonl				  covid-tweets-2020-08-10-2020-08-21.trigrams.txt
20000_tweets.txt				  GoogleNews-vectors-negative300.bin.gz
covid-tweets-2020-08-10-2020-08-21.tokenized.txt  stop_words.txt


Let's read the top 10 tweets from the file `20000_tweets.txt` and print them.

In [21]:
f = open("/content/drive/My Drive/tweets/20000_tweets.txt", "r")

line_count = 0
top_tweets = []
for tweet in f:
  print("### Tweet", line_count, "#####")
  print(tweet)

  top_tweets.append(tweet)
  line_count += 1
  if line_count >= 10:
    break
f.close()

### Tweet 0 #####
Covid-19 Economic Response: Cancel Student Loans by Executive Order. - Sign the Petition! https://t.co/BnPXWHv5cr via @Change

### Tweet 1 #####
Hey! The stock markets up! Fuck Trump and the Trumpublican Senate! https://t.co/4t6mgbaG2C

### Tweet 2 #####
@ProjectLincoln blame Trump for no sports because of Covid19 @MeidasTouch

### Tweet 3 #####
Do u guys think that reopening of the dinning in the restaurants of Lahore will bring any change in our lives? Nothing is going to change I think cause many people have learned cooking, painting and many new stuff during lockdown and I think we all will continue those things 1/2

### Tweet 4 #####
@NBCSAthletics Ya just knew the season wouldn’t go by without some bench clearing brawls... Covid or not. Behaviors can’t be changed because of rules, sadly

### Tweet 5 #####
You’re comparing apples and oranges. The NCAA has a lot more to balance than the NFL. https://t.co/oeMsbgeem7

### Tweet 6 #####
THANK YOU @POTUS Absolutely NO

# **Problem 1: Word Tokenizer**

Implement your own tokenizer for tweets without using libraries like `nltk`, `spacy` or any existing tokenizers. Tokenizer is a fundamental tool for processing any text data. The tokenizer should preserve punctuations.

The outputs for the top five tweets should approximately be as follows (it's fine to not exactly match this output)

```
['Covid-19', 'Economic', 'Response', ':', 'Cancel', 'Student', 'Loans', 'by', 'Executive', 'Order', '.', '-', 'Sign', 'the', 'Petition', '!', 'https://t.co/BnPXWHv5cr', 'via', '@Change']
['Hey', '!', 'The', 'stock', 'markets', 'up', '!', 'Fuck', 'Trump', 'and', 'the', 'Trumpublican', 'Senate', '!', 'https://t.co/4t6mgbaG2C']
['@ProjectLincoln', 'blame', 'Trump', 'for', 'no', 'sports', 'because', 'of', 'Covid19', '@MeidasTouch']
['Do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'Lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', '?', 'Nothing', 'is', 'going', 'to', 'change', 'I', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', ',', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'I', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@NBCSAthletics', 'Ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 'brawls', '...', 'Covid', 'or', 'not', '.', 'Behaviors', 'ca', 'n’t', 'be', 'changed', 'because', 'of', 'rules', ',', 'sadly']
```

A thing to note is that words like `wouldn't` and `you'll` become two tokens `would n't` and `you 'll`, after tokenization. Treating `n't` and  `'ll` as independent tokens is common in NLP tools. You can come up with that kind of list easily, e.g., `'s, 've`.


In [22]:
# Some rules:
# split by punctuations but not "@"
# split by whitespaces
# split by contractions
# keep hashtags

# import any relevant libraries here
import functools
import re
from urllib.parse import urlparse

PUNCTUATIONS = ["!", "\"", "$", "&", "\'", "(",")", "*",".",":", ";", "<", "=", ">", "?", "@", "[", "\\", "]", "^","_","`","{","|","}","~", ",", "..."]
CONTRACTIONS = {"'re", "'ve", "'d ", "'ll", "n’t", "'m", "'t", "’re", "’ve", "’d ", "’ll", "n’t", "’m", "’t", "'s", "’s"}

def is_url(url):
  parsed = urlparse(url)
  return bool(parsed.scheme) and bool(parsed.netloc)

def is_username(s):
  pattern = r'^@\S.*'
  return re.fullmatch(pattern, s)

def split_on_contractions(s):
  for contraction in CONTRACTIONS:
    if s.endswith(contraction):
      return [s.removesuffix(contraction), contraction]

  return [s]

def split_on_punctuations(s):
    if is_url(s) or is_username(s):
        return [s]

    words = []
    curr = []

    i = 0
    while i < len(s):
        # special case for triple dots
        if s[i:i+3] == "...":
            if curr:
                words.append("".join(curr))
                curr = []
            words.append("...")
            i += 3
        # special case for comma is separating numbers
        elif s[i] == ',' and i > 0 and i < len(s) - 1 and s[i-1].isdigit() and s[i+1].isdigit():
            curr.append(s[i])
            i += 1
        elif s[i] not in PUNCTUATIONS:
            curr.append(s[i])
            i += 1
        else:
            if curr:
                words.append("".join(curr))
                curr = []
            words.append(s[i])
            i += 1

    if curr:
        words.append("".join(curr))

    return [word for word in words if word]

def flatten(arr):
  return functools.reduce(lambda x, y: x + flatten(y), arr, []) if isinstance(arr, list) else [arr]

# a function to tokenize text into words
def tokenize(text):
  words = text.split()
  words = flatten([split_on_contractions(word) for word in words])
  words = flatten([split_on_punctuations(word) for word in words])
  words = [word for word in words if word]  # clear empty str

  return words

tokenized_top_tweets = [tokenize(tweet) for tweet in top_tweets]
for tokenized_tweet in tokenized_top_tweets:
  print(tokenized_tweet)

['Covid-19', 'Economic', 'Response', ':', 'Cancel', 'Student', 'Loans', 'by', 'Executive', 'Order', '.', '-', 'Sign', 'the', 'Petition', '!', 'https://t.co/BnPXWHv5cr', 'via', '@Change']
['Hey', '!', 'The', 'stock', 'markets', 'up', '!', 'Fuck', 'Trump', 'and', 'the', 'Trumpublican', 'Senate', '!', 'https://t.co/4t6mgbaG2C']
['@ProjectLincoln', 'blame', 'Trump', 'for', 'no', 'sports', 'because', 'of', 'Covid19', '@MeidasTouch']
['Do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'Lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', '?', 'Nothing', 'is', 'going', 'to', 'change', 'I', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', ',', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'I', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@NBCSAthletics', 'Ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 

# Problem 2: Post-process the tokenized tweets

Clean the tokenized tweets such that usernames are repalced with `@USER`, URLs are replaced with `URL`, punctuations are removed and words are lower-cased.

The output for the top ten tweets after cleaning should look something like this

```
['covid-19', 'economic', 'response', 'cancel', 'student', 'loans', 'by', 'executive', 'order', 'sign', 'the', 'petition', 'URL', 'via', '@USER']
['hey', 'the', 'stock', 'markets', 'up', 'fuck', 'trump', 'and', 'the', 'trumpublican', 'senate', 'URL']
['@USER', 'blame', 'trump', 'for', 'no', 'sports', 'because', 'of', 'covid19', '@USER']
['do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', 'nothing', 'is', 'going', 'to', 'change', 'i', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'i', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@USER', 'ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 'brawls', 'covid', 'or', 'not', 'behaviors', 'ca', 'n’t', 'be', 'changed', 'because', 'of', 'rules', 'sadly']
```

In [23]:
def replace_url(s):
    return "URL" if is_url(s) else s

def replace_username(s):
    return "@USER" if is_username(s) else s

# function to clean a tweet
def clean_a_tweet(tokenized_tweet):
  clean_tokenized_tweet = list(filter(lambda s: s not in PUNCTUATIONS,[replace_url(replace_username(s.lower())) for s in tokenized_tweet]))
  return clean_tokenized_tweet

anonymized_top_tweets = [clean_a_tweet(tokenized_tweet) for tokenized_tweet in tokenized_top_tweets]
for tokenized_tweet in anonymized_top_tweets:
  print(tokenized_tweet)

['covid-19', 'economic', 'response', 'cancel', 'student', 'loans', 'by', 'executive', 'order', '-', 'sign', 'the', 'petition', 'URL', 'via', '@USER']
['hey', 'the', 'stock', 'markets', 'up', 'fuck', 'trump', 'and', 'the', 'trumpublican', 'senate', 'URL']
['@USER', 'blame', 'trump', 'for', 'no', 'sports', 'because', 'of', 'covid19', '@USER']
['do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', 'nothing', 'is', 'going', 'to', 'change', 'i', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'i', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@USER', 'ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 'brawls', 'covid', 'or', 'not', 'behaviors', 'ca', 'n’t', 'be', 'changed', 'because', 'of', 'rules', 'sadly'

# Problem 3: Unigrams and Bigrams

Read **all** tweets from the file `20000_tweets.txt` and print the top 10 unigrams and bigrams. Consider only the English tweets. You have to use `tokenize` and `clean_a_tweet` function.

Output format

```
Top 10 Unigrams
@USER	16698
the	16666
URL	12610
to	11866
a	9845
....
....
....
....
....

Top 10 Bigrams
@USER @USER	7494
in the	1280
of the	1265
it ’s	860
a mask	793
....
....
....
....
....
```

Your numbers need not match the above word frequencies. Depending on your tokenizer, you may get different numbers

In [24]:
from collections import Counter

N = 10

with open("/content/drive/My Drive/tweets/20000_tweets.txt", "r") as fobj:
  lines = (line for line in fobj.readlines())

unigram_counts = Counter()
bigram_counts = Counter()

i = 0
for line in lines:
  tokenized_line = tokenize(line)
  cleaned_line = clean_a_tweet(tokenized_line)

  unigrams = cleaned_line
  bigrams = (f"{unigrams[i]} {unigrams[i + 1]}" for i in range(len(unigrams) - 1))

  for unigram in unigrams:
    unigram_counts[unigram] += 1

  for bigram in bigrams:
    bigram_counts[bigram] += 1

  i += 1

most_common_unigrams = unigram_counts.most_common(N)
most_common_bigrams = bigram_counts.most_common(N)

print("Top 10 Unigrams")
for k,v in most_common_unigrams:
  print(f"{k}    {v}")

print()

print("Top 10 Bigrams")
for k,v in most_common_bigrams:
  print(f'{k}    {v}')


Top 10 Unigrams
@USER    16631
the    16622
URL    12589
to    11854
a    9833
and    8513
of    7806
i    6497
is    6364
in    6289

Top 10 Bigrams
@USER @USER    7494
in the    1280
of the    1263
it ’s    848
a mask    781
this is    727
the pandemic    719
@USER i    718
wear a    691
URL URL    681


# Problem 4: How frequent are the following unigrams and bigrams?

```
covid
coronavirus
republicans
democrats
social distancing
wear mask
no mask
donald trump
joe biden
```

Output:
```
covid	4437
coronavirus	1368
...
...
social distancing	568
wear mask	19
...
...
...
```


In [25]:
woi = [
  "covid",
  "coronavirus",
  "republicans",
  "democrats",
  "social distancing",
  "wear mask",
  "no mask",
  "donald trump",
  "joe biden"
]

for word in woi:
  counter = unigram_counts if len(word.split()) == 1 else bigram_counts
  print(f"{word}    {counter[word]}")

covid    4299
coronavirus    1354
republicans    57
democrats    162
social distancing    553
wear mask    19
no mask    19
donald trump    61
joe biden    22
